In [1]:
# import required libraries 
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

from numpy import array

from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import gc
import statsmodels.formula.api as sm

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

get_ipython().run_line_magic('matplotlib', 'inline')

In [39]:
df1 = pd.read_csv("/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", skipinitialspace = True)
df2 = pd.read_csv("/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", skipinitialspace = True)


dataset = pd.concat([df1,df2]).drop_duplicates(keep=False)
dataset.reset_index(drop=True, inplace = True)



In [40]:
col_names = [col.replace(' ', '') for col in dataset.columns]
dataset.columns = col_names
dataset.head()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [41]:
label_names = dataset['Label'].unique()

import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

(['BENIGN', 'DDoS', 'PortScan'], 3)

In [42]:
labels = dataset['Label'].unique()

for i in range(0,len(label_names)):
    dataset['Label'] = dataset['Label'].replace({labels[i] : label_names[i]})
    
dataset['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan'], dtype=object)

In [43]:
dataset.dropna(inplace=True)

In [44]:
labl = dataset[['Label']]
dataset = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [45]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [46]:
dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)

In [47]:
dataset.dropna(inplace=True)

In [48]:
dataset.columns

Index(['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       '

In [49]:
dataset.Label.value_counts()

BENIGN      212718
DDoS        128005
PortScan     57305
Name: Label, dtype: int64

In [50]:
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [52]:
labels=pd.DataFrame()
labels["Label"] = dataset['Label']
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [53]:
scaler = RobustScaler() #scade din date media acestora si le scaleaza 
scaler.fit(features)

features = scaler.transform(features)

In [54]:
labels["Label"].value_counts()

BENIGN      212718
DDoS        128005
PortScan     57305
Name: Label, dtype: int64

In [ ]:
# df.drop_duplicates(keep='first', inplace = True)
# df.dropna(inplace=True,axis=1) 
# print("Read {} rows.".format(len(df)))

In [55]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

LE.fit(labels["Label"])
labels["Label"] = LE.transform(labels["Label"])

In [56]:
labels.value_counts()


Label
0        212718
1        128005
2         57305
dtype: int64

In [57]:
d = LE.inverse_transform(labels["Label"])
d = pd.Series(d)
d.unique()

array(['BENIGN', 'DDoS', 'PortScan'], dtype=object)

In [58]:
from imblearn.over_sampling import SMOTE #dublifica valori in tipurile de tipurile de clase care au mai putine pentru a avea un set de date balansat
smt = SMOTE()
features, labels = smt.fit_resample(features, labels)

In [59]:
labels.value_counts()


Label
0        212718
1        212718
2        212718
dtype: int64

In [60]:
from sklearn.model_selection import train_test_split

# The next step is to split training and testing data. For this we will use sklearn function train_test_split().

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=.2)

features_train.shape, features_test.shape, labels_train.shape, labels_test.shape

((510523, 78), (127631, 78), (510523, 1), (127631, 1))

In [63]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    
    tf.keras.layers.Flatten(input_shape=(78,)),
    tf.keras.layers.Dense(67, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(67, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [71]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [69]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="weights.mapt.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
metrics_plot = model.fit(features_train,
                         labels_train,
                         epochs = 25, 
                         verbose=1 ,
                         callbacks=callbacks_list,
                         validation_data=(features_test,labels_test))

Epoch 1/25
15935/15954 [============================>.] - ETA: 0s - loss: 0.1039 - accuracy: 0.9499
Epoch 1: accuracy improved from -inf to 0.94992, saving model to weights.mapt.hdf5
15954/15954 [==============================] - 43s 3ms/step - loss: 0.1039 - accuracy: 0.9499 - val_loss: 0.0886 - val_accuracy: 0.9549
Epoch 2/25
15942/15954 [============================>.] - ETA: 0s - loss: 0.0903 - accuracy: 0.9554
Epoch 2: accuracy improved from 0.94992 to 0.95541, saving model to weights.mapt.hdf5
15954/15954 [==============================] - 42s 3ms/step - loss: 0.0904 - accuracy: 0.9554 - val_loss: 0.0764 - val_accuracy: 0.9540
Epoch 3/25
15950/15954 [============================>.] - ETA: 0s - loss: 0.0865 - accuracy: 0.9576
Epoch 3: accuracy improved from 0.95541 to 0.95761, saving model to weights.mapt.hdf5
15954/15954 [==============================] - 42s 3ms/step - loss: 0.0865 - accuracy: 0.9576 - val_loss: 0.0741 - val_accuracy: 0.9606
Epoch 4/25
15944/15954 [=============

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 6, 8

results = pd.read_csv("history.csv", skipinitialspace = True)
fig, ax = plt.subplots(2,1)
fig.tight_layout()
figure(figsize=(10, 6), dpi=150)
ax[0].plot(results['accuracy'], color='b', label="Train accuracy")
ax[0].plot(results['val_accuracy'], color='r', label="Validation data accuracy",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

ax[1].plot(results['loss'], color='b', label="Loss")
ax[1].plot(results['val_loss'], color='r',label="Validation data loss")
legend = ax[1].legend(loc='best', shadow=True)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')



FileNotFoundError: [Errno 2] No such file or directory: 'history.csv'